In [6]:
#importing dependencies:
%matplotlib inline

#importing the python libraries:
import math, time, random, datetime

#data manipulation:
import numpy as np
import pandas as pd

#for visualization:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-whitegrid")

#preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize